Connected to project (Python 3.12.0)

Restarted project (Python 3.12.0)

In [1]:
%load_ext autoreload
%autoreload 1


In [2]:
#!/usr/bin/env python3

import os
import time
import json
import threading
import configparser
import datetime
import random
%aimport schema_scanner
%aimport query_generator
%aimport query_mutator_sequential
%aimport query_generator_subqueries_with_graph
%aimport testing


#!/usr/bin/env python3

import os
import time
import json
import threading
import configparser
import datetime

from schema_scanner import *
from query_generator import *
from query_mutator_sequential import *
from query_generator_subqueries_with_graph import *

import graph_tool.all as gt
from testing import *


In [3]:

if __name__ == "__main__":
    print("=====GraphGenie=====")
    config = configparser.ConfigParser()
    config.read('graphgenie.ini')
    graphdb = config['default']['graphdb']
    language = config['default']['language']
    ip = config['default']['ip']
    port = int(config['default']['port'])
    username = config['default']['username']
    password = config['default']['password']

    
    from neo4j import GraphDatabase
    test = Neo4jTesting()
    graph_full = gt.Graph(directed=True)



    schema_scanner = Neo4jSchemaScanner(ip, port, username, password)
    node_labels, edge_labels, node_properties, connectivity_matrix, properties_types,graph_full = schema_scanner.scan(graph_full)
    


Transaction failed and will be retried in 0.9778652078668851s (Couldn't connect to 127.0.0.1:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))


=====GraphGenie=====


Transaction failed and will be retried in 2.0757543875242033s (Couldn't connect to 127.0.0.1:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Transaction failed and will be retried in 4.380559950270374s (Couldn't connect to 127.0.0.1:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Transaction failed and will be retried in 6.6396594802766975s (Couldn't connect to 127.0.0.1:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Transaction failed and will be retried in 15.413284627943348s (Couldn't connect to 127.0.0.1:7687 (resolved to ()):
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))


Node count: 28863
Edge count: 332522
Node labels: ['Actor', 'Director', 'Genre', 'Movie', 'Person', 'User']
Edge labels: ['ACTED_IN', 'DIRECTED', 'IN_GENRE', 'RATED']
each_node_label: ['Actor', 'Director', 'Genre', 'Movie', 'Person', 'User']
properties_types:{'bornIn': 'STRING NOT NULL', 'id': 'INTEGER NOT NULL', 'born': 'DATE NOT NULL', 'died': 'DATE NOT NULL', 'tmdbId': 'STRING NOT NULL', 'imdbId': 'STRING NOT NULL', 'name': 'STRING NOT NULL', 'url': 'STRING NOT NULL', 'bio': 'STRING NOT NULL', 'poster': 'STRING NOT NULL', 'runtime': 'INTEGER NOT NULL', 'revenue': 'INTEGER NOT NULL', 'budget': 'INTEGER NOT NULL', 'imdbRating': 'FLOAT NOT NULL', 'released': 'STRING NOT NULL', 'countries': 'LIST<STRING NOT NULL> NOT NULL', 'languages': 'LIST<STRING NOT NULL> NOT NULL', 'plot': 'STRING NOT NULL', 'imdbVotes': 'INTEGER NOT NULL', 'year': 'INTEGER NOT NULL', 'movieId': 'STRING NOT NULL', 'title': 'STRING NOT NULL', 'userId': 'STRING NOT NULL'}
A D G M P U 
0 0 0 1 0 0 
0 0 0 1 0 0 
0 0 0 

100%|██████████| 28863/28863 [00:00<00:00, 784139.73it/s]


Querying vertices and edges (Can take a while))
Adding vertices and edges (with their properties) (Can take a while))


100%|██████████| 166261/166261 [00:01<00:00, 122755.39it/s]


Graph filled


In [4]:
# print("CONNECTIVITY MATRIX:",connectivity_matrix);
random_cypher_generator = RandomCypherGenerator_subqueries_with_graph(node_labels, edge_labels, node_properties, connectivity_matrix,properties_types,graph_full)
cypher_query_mutator = CypherQueryMutatorSequential(node_labels, edge_labels, node_properties, connectivity_matrix)


INIT:['Actor', 'Director', 'Genre', 'Movie', 'Person', 'User'], edge_labels['ACTED_IN', 'DIRECTED', 'IN_GENRE', 'RATED'],connectivity_matrix[[0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0]],property_types_dict{'bornIn': 'STRING NOT NULL', 'id': 'INTEGER NOT NULL', 'born': 'DATE NOT NULL', 'died': 'DATE NOT NULL', 'tmdbId': 'STRING NOT NULL', 'imdbId': 'STRING NOT NULL', 'name': 'STRING NOT NULL', 'url': 'STRING NOT NULL', 'bio': 'STRING NOT NULL', 'poster': 'STRING NOT NULL', 'runtime': 'INTEGER NOT NULL', 'revenue': 'INTEGER NOT NULL', 'budget': 'INTEGER NOT NULL', 'imdbRating': 'FLOAT NOT NULL', 'released': 'STRING NOT NULL', 'countries': 'LIST<STRING NOT NULL> NOT NULL', 'languages': 'LIST<STRING NOT NULL> NOT NULL', 'plot': 'STRING NOT NULL', 'imdbVotes': 'INTEGER NOT NULL', 'year': 'INTEGER NOT NULL', 'movieId': 'STRING NOT NULL', 'title': 'STRING NOT NULL', 'userId': 'STRING NOT NULL'},


NameError: name 'CypherQueryMutator' is not defined

In [5]:
    # test.testing(random_cypher_generator, cypher_query_mutator)
    # random_cypher_generator.init()
    
    # for i in range(10):
    #     print("\n","="*100)
    #     base_query = random_cypher_generator.random_query_generator()
    #     print()
    #     print(base_query)
    #     print()
    #     print(random_cypher_generator.symbols)
    #     print(random_cypher_generator.node_symbols)
    #     print(random_cypher_generator.name_label_dict)
    #     print(random_cypher_generator.node_properties)
    #     print(f"property_to_test:{random_cypher_generator.property_to_test}")
    #     print(f"number of nested: {random_cypher_generator.number_nested_predicates}")
    #      #input()
    
    # # for i in range(10):
    # #     equivalent_queries, equivalent_rules_eval = cypher_query_mutator.generate_equivalent_queries(base_query)
    # #     print(equivalent_queries)
    # #     print()
    # #     print(equivalent_rules_eval)
    # #     print("=================================")

In [6]:
test.current_rules_eval = [0, 0, 0]
test.executed_allquery = 0
test.executed_query_num = 0
test.executed_base_query_num = 0
test.detected_bug_num = 0

In [7]:
base_query = random_cypher_generator.random_query_generator()
base_query_result, base_query_time = test.execute_ret_result_time(query=base_query, log_str="[BASE QUERY]")
print(base_query)
print(base_query_result)


MATCH (id18935:Actor)-[:ACTED_IN]->(id1178:Movie)-[:IN_GENRE]->(id16:Genre) WHERE True RETURN DISTINCT id18935   LIMIT 5
{'bornIn': 'London, England, UK', 'tmdbId': '15440', 'imdbId': '0364813', 'born': neo4j.time.Date(1961, 8, 24), 'name': 'Jared Harris', 'bio': 'Jared Francis Harris (born 24 August 1961) is an English actor, well-known for his portrayal of Lane Pryce on the AMC series Mad Men...', 'id': 18920, 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/mGODbYpkR1NKBOV3rtTfZ7EWoIl.jpg', 'url': 'https://themoviedb.org/person/15440'}


In [8]:
%aimport query_mutator_sequential
from query_mutator_sequential import CypherQueryMutatorSequential

In [9]:
cypher_query_mutator = CypherQueryMutatorSequential(node_labels, edge_labels, node_properties, connectivity_matrix,graph_full)

In [10]:
base_query = random_cypher_generator.random_query_generator()
print(base_query)

COULD NOT FIND A PATH
COULD NOT FIND A PATH
MATCH (id1540:Movie)-[:IN_GENRE]->(id9:Genre) WHERE ( id1540.id = 1540 ) AND ( id9.id = 9 ) AND EXISTS { MATCH (id10515:Person)-[:ACTED_IN]->(id2716:Movie)-[:IN_GENRE]->(id162:Genre) WHERE ( id10515.id = 10515 ) AND ( id2716.id = 2716 ) AND ( id162.id = 162 ) AND EXISTS { MATCH (id19884:Actor)-[:ACTED_IN]->(id1490:Movie)-[:IN_GENRE]->(id4:Genre) WHERE ( id19884.id = 19884 ) AND ( id1490.id = 1490 ) AND ( id4.id = 4 ) AND EXISTS { MATCH (id7549:Movie)-[:IN_GENRE]->(id10:Genre) WHERE ( id7549.id = 7549 ) AND ( id10.id = 10 ) AND EXISTS { MATCH (id29319:Director)-[:DIRECTED]->(id5484:Movie)-[:IN_GENRE]->(id10:Genre) WHERE ( id29319.id = 29319 ) AND ( id5484.id = 5484 ) AND ( id10.id = 10 ) AND EXISTS { (id9811:User)-[:RATED]->(id334:Movie)-[:IN_GENRE]->(id33:Genre) WHERE ( id9811.id = 9811 ) AND ( id334.id = 334 ) AND ( id33.id = 33 ) } } } } } RETURN id1540 ORDER BY NULL SKIP 0 LIMIT 5


In [11]:
cypher_query_mutator.views_ids = []

In [12]:
base_query = 'MATCH (id32579:Actor)-[:ACTED_IN]->(id4227:Movie)-[:IN_GENRE]->(id13:Genre) WHERE ( id32579.id = 32579 ) AND ( id4227.id = 4227 ) AND ( id13.id = 13 ) AND EXISTS { OPTIONAL MATCH (id1820:Movie)-[:IN_GENRE]->(id13:Genre) WHERE ( id1820.id = 1820 ) AND ( id13.id = 13 ) AND EXISTS { OPTIONAL MATCH (id17332:Person)-[:ACTED_IN]->(id5882:Movie)-[:IN_GENRE]->(id49:Genre) WHERE ( id17332.id = 17332 ) AND ( id5882.id = 5882 ) AND ( id49.id = 49 ) AND EXISTS { (id12688:Actor)-[:ACTED_IN]->(id3905:Movie)-[:IN_GENRE]->(id33:Genre) WHERE ( id12688.id = 12688 ) AND ( id3905.id = 3905 ) AND ( id33.id = 33 ) } } } RETURN id4227  SKIP 0 LIMIT 5'

In [13]:
base_query = random_cypher_generator.random_query_generator()
print(base_query)

MATCH (id10435:Person)-[:ACTED_IN]->(id5266:Movie)-[:IN_GENRE]->(id16:Genre) WHERE ( id10435.id = 10435 ) AND ( id5266.id = 5266 ) AND ( id16.id = 16 ) AND EXISTS { MATCH (id21711:Actor)-[:ACTED_IN]->(id4830:Movie)-[:IN_GENRE]->(id6:Genre) WHERE ( id21711.id = 21711 ) AND ( id4830.id = 4830 ) AND ( id6.id = 6 ) AND EXISTS { MATCH (id18419:Person)-[:DIRECTED]->(id235:Movie)-[:IN_GENRE]->(id3:Genre) WHERE ( id18419.id = 18419 ) AND ( id235.id = 235 ) AND ( id3.id = 3 ) AND EXISTS { (id16314:Actor)-[:ACTED_IN]->(id3619:Movie)-[:IN_GENRE]->(id4:Genre) WHERE ( id16314.id = 16314 ) AND ( id3619.id = 3619 ) AND ( id4.id = 4 ) } } } RETURN id16 ORDER BY NULL SKIP 0 LIMIT 4


In [14]:
# this_level_query,subquery = cypher_query_mutator.parse_subquerie(base_query)
# print(base_query)
# print()
# print(this_level_query)
# print()
# print(subquery)

In [15]:
cypher_query_mutator.get_all_transactions(base_query)


MATCH (id10435:Person)-[:ACTED_IN]->(id5266:Movie)-[:IN_GENRE]->(id16:Genre) WHERE ( id10435.id = 10435 ) AND ( id5266.id = 5266 ) AND ( id16.id = 16 ) 
CREATE (view0:view0)-[:contains]->(id10435)-[:ACTED_IN]->(id5266)-[:IN_GENRE]->(id16)  

 MATCH (view0)-[:contains]->() (()-[]->()){0,1000} (id21711:Actor)-[:ACTED_IN]->(id4830:Movie)-[:IN_GENRE]->(id6:Genre) WHERE ( id21711.id = 21711 ) AND ( id4830.id = 4830 ) AND ( id6.id = 6 ) 
CREATE (view1:view1)-[:contains]->(id21711)-[:ACTED_IN]->(id4830)-[:IN_GENRE]->(id6)  

 MATCH (view1)-[:contains]->() (()-[]->()){0,1000} (id18419:Person)-[:DIRECTED]->(id235:Movie)-[:IN_GENRE]->(id3:Genre) WHERE ( id18419.id = 18419 ) AND ( id235.id = 235 ) AND ( id3.id = 3 ) 
CREATE (view2:view2)-[:contains]->(id18419)-[:DIRECTED]->(id235)-[:IN_GENRE]->(id3)  

MATCH (view2)-[:contains]->() (()-[]->()){0,1000}  (id16314:Actor)-[:ACTED_IN]->(id3619:Movie)-[:IN_GENRE]->(id4:Genre) WHERE ( id16314.id = 16314 ) AND ( id3619.id = 3619 ) AND ( id4.id = 4 ) 
CRE

In [16]:
cypher_query_mutator.get_all_transactions_delete()

['MATCH (view:view0) DETACH DELETE view',
 'MATCH (view:view1) DETACH DELETE view',
 'MATCH (view:view2) DETACH DELETE view',
 'MATCH (view:view3) DETACH DELETE view']